In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} </style>"))

In [2]:
from tensorflow import keras

In [3]:
import tensorflow

In [4]:
tensorflow.__version__

'2.0.0'

# DNN

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

redwine = pd.read_csv("winequality-red.csv", sep=';')
redwine_X = redwine.iloc[:, :-1]
redwine_y = redwine.iloc[:, -1]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(redwine_y.to_numpy().reshape(-1,1))

y_onehot = enc.transform(redwine_y.to_numpy().reshape(-1,1))

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(redwine_X, y_onehot,
                                                    test_size=0.3,
                                                    random_state=1)
x = tf.placeholder(tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 6])

W1 = tf.Variable(tf.zeros([11, 20]))
b1 = tf.Variable(tf.zeros([20]))
h1 = tf.nn.tanh(tf.matmul(x, W1)+b1)

W2 = tf.Variable(tf.zeros([20, 15]))
b2 = tf.Variable(tf.zeros([15]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

Wo = tf.Variable(tf.zeros([15, 6]))
bo = tf.Variable(tf.zeros([6]))
ho = tf.nn.softmax(tf.matmul(h2, Wo)+bo)

cross_entropy = -tf.reduce_sum(y * tf.log(ho), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)

training = tf.train.AdamOptimizer(0.01).minimize(loss)
#training = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(100001):
    _, loss_value = sess.run([training, loss],
                              feed_dict={x: train_X,
                                         y: train_y.toarray()})
    if i % 10000 == 0:
        print('Step %d, %.5f' %(i, loss_value))

Instructions for updating:
non-resource variables are not supported in the long term


FileNotFoundError: [Errno 2] File winequality-red.csv does not exist: 'winequality-red.csv'

In [ ]:
pred = sess.run(ho, feed_dict={x:test_X})
pd.crosstab(np.argmax(test_y.toarray(), axis=1)+3,
            np.argmax(pred, axis=1)+3)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(test_y.toarray(), axis=1),
               np.argmax(pred, axis=1))

# CNN

### Implementing the Regression

이미지와 정답 레이블을 담을 placeholder와 학습 결과인 가중치(weight)와 바이어스(bias)를 담을 Variable을 정의하고 Softmax Regression 모델을 정의합니다.

In [ ]:
""" placeholder 정의 : 데이터가 들어 갈 곳
이미지와 정답 레이블용 2차원 tensor를 만든다.
None은 어떤 length도 가능함을 의미한다. """
# 이미지 데이터용 placeholder
x = tf.placeholder(tf.float32, [None, 784])
# 정답 레이블용 placeholder
y_ = tf.placeholder(tf.float32, [None, 10])

""" Variable 정의 : 학습 결과가 저장될 가중치(weight)와 바이어스(bias) """
# 0으로 초기화 함
W = tf.Variable(tf.zeros([784, 10])) # w는 784차원의 이미지 벡터를 곱해, 10차원(one hot encoding된 0~9)의 결과를 내기 위한 것
b = tf.Variable(tf.zeros([10]))      # b는 결과에 더해야 하므로 10차원

""" 모델 정의 : Softmax Regression
10개의 값 중 가장 확률이 높은 것을 고르기 위해 Softmax 사용 """
y = tf.nn.softmax(tf.matmul(x, W) + b)



### Training

모델 훈련에 필요한 Loss 함수와 학습율(Learning Rate)을 정의하고 100개씩 샘플링하여 모델을 1000회 학습시킵니다. 샘플링 데이터 수를 늘리면 정확도가 올라갈 수는 있지만 학습 시간이 증가합니다. 랜덤 샘플링한 작은 배치로 학습하는 것을 Stochastic Training이라고 하며, 비용이 싸고 비슷한 결과를 낼 수 있어서 많이 사용됩니다.

In [ ]:
""" 모델 훈련 """
# Loss 함수 정의
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# learning rate을 0.5로 정의
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# 세션 시작 전에 모든 변수를 초기화
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

# 100개씩 샘플링하여 1000회 학습 진행
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)  # 학습 데이터셋에서 무작위로 샘플링한 100개의 데이터로 구성된 'batch'를 가져옴
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})    # placeholder x, y_에 샘플링된 batch_xs, batch_ys를 공급함

## CNN 모델
본 예제 코드는 TensorFlow에서 TensorFlow 전문가를 위해 제공하는 MNIST Deep Learning 고급 예제를 다루고 있습니다. Deep Learning의 하나인 CNN(Convolutional Neural Network) 모델을 만들어 훈련시킨 후 label을 예측하고 모델의 정확도를 구해볼 것입니다.

### 가중치(weight)와 바이어스(bias) 초기화
대칭성을 깨뜨리고 gradient가 0이 되는 것을 막기 위해 약간의 noise를 줘서 가중치를 초기화하고, ReLU neuron을 사용하기 때문에 죽은 뉴런이 되는 것을 막기 위해 바이어스를 작은 양수값인 0.1로 초기화합니다.

In [6]:
""" 가중치 초기화 """
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

""" Bias 초기화 """
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution과 Pooling 정의
아래 코드를 통해 Convolution Layer의 stride를 1로 설정하고, 출력 크기가 입력과 같게 되도록 하기 위해 패딩을 0으로 설정합니다. 풀링은 2x2 크기의 맥스 풀링을 적용하고 stride를 2로 설정합니다.

In [7]:
""" Convolution 정의 """
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

""" Pooling 정의 """
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### Convolutional Layer 정의
아래 코드는 28x28 이미지가 두 번의 Convolutional Layer를 거치면서 7x7 이미지로 변하는 모습을 보여주고 있습니다. <br>
즉, 28x28 이미지가 첫 번째 Convolutional Layer(3x3 필터, 스트라이드 1)를 거치면서 28x28가 되고 첫 번째 Max Pooling 상하좌우 패딩 후 28x28, 스트라이드 2를 거치면서 14x14가 됩니다.<br>
그리고 두 번째 Convolutional Layer(3x3 필터, 스트라이드 1)를 거치면서 14x14이 되고 두 번째 Max Pooling 상하좌우 패딩 후 14x14, 스트라이드 2를 거치면서 7x7이 됩니다.

In [8]:
# 입력 데이터를 4D 텐서로 재정의
# 두 번째/세 번째 파라미터는 이미지의 가로/세로 길이
# 마지막 파라미터 컬러 채널의 수는 흑백 이미지이므로 1임
x_image = tf.reshape(x, [-1,28,28,1])

""" First Convolutional Layer 정의 """
# 가중치 텐서 정의(patch size, patch size, input channel, output channel).
# 3x3의 윈도우(patch라고도 함) 크기를 가지는 32개의 feature(kernel, filter)를 사용
# 흑백 이미지이므로 input channel은 1임
W_conv1 = weight_variable([3, 3, 1, 32])
# 바이어스 텐서 정의
b_conv1 = bias_variable([32])
# x_image와 가중치 텐서에 합성곱을 적용하고, 바이어스을 더한 뒤 ReLU 함수를 적용
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 출력값을 구하기 위해 맥스 풀링을 적용
h_pool1 = max_pool_2x2(h_conv1)

""" Second Convolutional Layer 정의 """
# 가중치 텐서 정의(patch size, patch size, input channel, output channel)
# 3x3의 윈도우(patch라고도 함) 크기를 가지는 64개의 feature를 사용
# 이전 레이어의 output channel의 크기가 32가 여기에서는 input channel이 됨
W_conv2 = weight_variable([3, 3, 32, 64])
# 바이어스 텐서 정의
b_conv2 = bias_variable([64])
# First Convolutional Layer의 출력값인 h_pool1과 가중치 텐서에 합성곱을 적용하고, 바이어스을 더한 뒤 ReLU 함수를 적용
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
# 출력값을 구하기 위해 맥스 풀링을 적용
h_pool2 = max_pool_2x2(h_conv2)

""" 완전 연결 레이어(Fully-Connected Layer) 정의 """
#  7×7 크기의 64개 필터. 임의로 선택한 뉴런의 개수(여기서는 1024)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

""" Dropout 정의 """
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

""" 최종 소프트맥스 계층 정의 """
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

NameError: name 'x' is not defined